In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

* prepare all the things 

In [2]:
tf.reset_default_graph()
# sess = tf.InteractiveSession()
IMAGEPATH = "/home/breakpoint/software/caffe/data/flickr_style/images/*.jpg"

In [ ]:
# path to images
IMAGEPATH = "/home/breakpoint/software/caffe/data/flickr_style/images/*.jpg"

# filenames' queue of images
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once(IMAGEPATH))

# how to load image
image_reader = tf.WholeFileReader()

key,image_file = image_reader.read(filename_queue)

image = tf.image.decode_jpeg(image_file)

# check the shape of image
# conclude that the shape of every image is not the same 


# so resize image first
height = 255
width = 255

resized_image = tf.image.resize_images(image,width,height)

# resized_image.set_shape((width, height, 3))
image_shape = tf.shape(resized_image)
# some arguements to set
# batch_size = 1

# min_after_dequeue = 128
# capacity = min_after_dequeue + 3 * batch_size

# num_preprocess_threads = 3
# image_batch = tf.train.shuffle_batch(
#     [resized_image],
#     batch_size=batch_size,
#     capacity=capacity,
#     num_threads=num_preprocess_threads,
#     min_after_dequeue=min_after_dequeue)


# run
init = tf.initialize_all_variables()

with tf.Session() as sess:
    
    init.run()

    coord = tf.train.Coordinator()

    threads = tf.train.start_queue_runners(coord=coord)

#     image_tensor = resized_image.eval()
    resized_image_tensor,shape_tensor,key_tensor = sess.run([resized_image,
                                        image_shape,
                                        key])
    print(shape_tensor)
    print(key_tensor)
    print(resized_image_tensor.shape)
#     image_tensor = sess.run(resized_image)
    plt.imshow(Image.fromarray(resized_image_tensor))
#     image_batch_tensor = image_batch.eval()

#     image_batch_tensor = sess.run(image_batch)
#     print(image_batch_tensor.shape)

    # reduce threads
    coord.request_stop()
    coord.join(threads)

In [3]:
def generate_image(path2image,isBatch=False,resize=False):
    IMAGEPATH = path2image

    # filenames' queue of images
    filename_queue = tf.train.string_input_producer(
        tf.train.match_filenames_once(IMAGEPATH))

    # how to load image
    image_reader = tf.WholeFileReader()

    key,image_file = image_reader.read(filename_queue)

    image = tf.image.decode_jpeg(image_file)

    # check the shape of image
    # conclude that the shape of every image is not the same 


    # so resize image first
    height = 255
    width = 255
    
    if(resize):
        
        resized_image = tf.image.resize_images(image,width,height)

        resized_image.set_shape((width, height, 3))
        
        return key,resized_image
    
    if(isBatch):
        
        # some arguements to set
        batch_size = 1
        
        min_after_dequeue = 128
        capacity = min_after_dequeue + 3 * batch_size

        num_preprocess_threads = 3
        image_batch = tf.train.shuffle_batch(
            [resized_image],
            batch_size=batch_size,
            capacity=capacity,
            num_threads=num_preprocess_threads,
            min_after_dequeue=min_after_dequeue)
        return key,image_batch
        
    return key,image


In [ ]:
def run(path2image,isBatch=False,resize=False,debug=True):
    # run
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
    
        init.run()

        coord = tf.train.Coordinator()

        threads = tf.train.start_queue_runners(coord=coord)

        key,image_batch = generate_image(path2image)
        image_name,image_batch_tensor = sess.run([key,image_batch])
        
#         if it is should show sth for debugging
        if(debug):
            print(image_batch_tensor.shape)
            if(not isBatch):
                print('display image:%s'%image_name)
#                 note here image is not batched
#                 plt.imshow(Image.fromarray(image_batch_tensor))
                

        # reduce threads
        coord.request_stop()
        coord.join(threads)

In [ ]:
# test function listed before
run(IMAGEPATH)

In [4]:
_,image = generate_image(IMAGEPATH)

In [ ]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    image_tensor = sess.run([image])
    print(image_tensor.shape)